In [ ]:
import numpy as np
import pandas as pd
import json
import re
import os
from google.colab import drive
import random
import torch

In [ ]:
from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN')

In [ ]:
!pip install -q transformers
!pip install -q huggingface_hub
!pip install -q datasets
!pip install -q accelerate
!pip install -q peft
!pip install -q bitsandbytes
# !pip install -q trl
# !pip install --upgrade -q trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.3 MB/s eta 0:00:00


In [ ]:
if torch.cuda.is_available():
    torch.backends.cuda.matmul.allow_tf32 = True

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
model_id="TinyLlama/TinyLlama_v1.1_math_code"

In [ ]:
# Load your dataset (example with input-output pairs)
from datasets import load_dataset
from transformers import AutoTokenizer

train_dataset = load_dataset('csv', data_files=['/content/drive/MyDrive/Fine-tuning-llms/qe_input_dataset.csv'],\
                       column_names=['input','output'], split='train[:90%]')
eval_dataset = load_dataset('csv', data_files=['/content/drive/MyDrive/Fine-tuning-llms/qe_input_dataset.csv'],\
                       column_names=['input','output'], split='train[98%:]')

# Load the tokenizer for your model (LLaMA tokenizer, for example)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# Define a function to tokenize your dataset
def preprocess_function(examples):
    texts=[]
    for inputs,outputs in zip(examples['input'],examples['output']):
        input_text = 'user: '+ inputs + 'assistant: '+ outputs + tokenizer.eos_token
        texts.append(input_text)

    tokenized_inputs = tokenizer(texts, padding="max_length", truncation=True, max_length=2048,return_tensors = "pt")
    return tokenized_inputs

# Apply the preprocessing function to your dataset
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

# Remove the original columns (instruction, output) as we now have input_ids and labels
train_dataset = train_dataset.remove_columns(['input', 'output'])
eval_dataset = eval_dataset.remove_columns(['input', 'output'])

Generating train split: 0 examples [00:00, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Map:   0%|          | 0/6715 [00:00<?, ? examples/s]

Map:   0%|          | 0/149 [00:00<?, ? examples/s]

In [ ]:
len(eval_dataset),len(train_dataset)

(149, 6715)

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [ ]:
from transformers import AutoModelForCausalLM, set_seed

base_model = AutoModelForCausalLM.from_pretrained(
        "TinyLlama/TinyLlama_v1.1_math_code",
        device_map = 'auto')


base_model.config.use_cache = False
set_seed(123)

config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.01,
    task_type="CAUSAL_LM",
)

In [ ]:
from peft import prepare_model_for_kbit_training, get_peft_model
from transformers import TrainingArguments
from transformers import BitsAndBytesConfig
import torch

# # bitsandbytes
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
device_map = "auto"

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
print(compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant
)

model = prepare_model_for_kbit_training(base_model)
model = get_peft_model(model, peft_config)

train_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Fine-tuning-llms/test_output',
    evaluation_strategy="steps",
    max_steps = 120,
    weight_decay=0,
    save_strategy='steps',
    save_steps=20,
    warmup_ratio=0.09,
    learning_rate=0.002,
    max_grad_norm=0.3,
    gradient_checkpointing=True,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    per_device_eval_batch_size=8,
    # num_train_epochs=1,
    logging_steps=20,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    load_best_model_at_end=True,

)
from transformers import Trainer
trainer = Trainer(
        model=model,
        args=train_args,
        tokenizer=tokenizer,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
    )

torch.float16


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
def print_trainable_parameters(args, model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || "
        f"all params: {all_param} || "
    )

# Verifying the datatypes and parameter counts before training.
print_trainable_parameters(train_args, model)
dtypes = {}
for _, p in model.named_parameters():
    dtype = p.dtype
    if dtype not in dtypes: dtypes[dtype] = 0
    dtypes[dtype] += p.numel()
total = 0
for k, v in dtypes.items():
    total+= v
for k, v in dtypes.items():
    print(k, v, v/total)

trainable params: 26320896 || all params: 1126369280 || 
torch.float32 1126369280 1.0


In [ ]:
26320896/1126369280

0.023367910033910016

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss,Validation Loss
20,0.772500,0.575185


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:227: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss,Validation Loss
20,0.772500,0.575185
40,0.443400,0.335048
60,0.276600,0.226732
80,0.203000,0.188361
100,0.185800,0.172481
120,0.168800,0.167491


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:227: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:227: User

TrainOutput(global_step=120, training_loss=0.3417005797227224, metrics={'train_runtime': 4610.2524, 'train_samples_per_second': 0.208, 'train_steps_per_second': 0.026, 'total_flos': 1.25140985708544e+16, 'train_loss': 0.3417005797227224, 'epoch': 0.14285714285714285})

In [ ]:
model.save_pretrained("/content/drive/MyDrive/Fine-tuning-llms/test_output/model_tiny_32_120/")
peft_config.save_pretrained("/content/drive/MyDrive/Fine-tuning-llms/test_output/model_tiny_32_120/")

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:227: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


In [ ]:
from peft import LoraConfig, TaskType, AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained("/content/drive/MyDrive/Fine-tuning-llms/test_output/model_tiny_32_120/")
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama_v1.1_math_code")

In [ ]:
model = model.to("cuda")
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.01, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Lin

In [ ]:
inputs = tokenizer("Generate an input file for NaCl for Quantum Espresso calculations.", return_tensors="pt")

In [ ]:
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=2048)
print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

Generate an input file for NaCl for Quantum Espresso calculations.  Cell parameters are       0.0000000000       0.0000000000       0.0000000000
      0.0000000000       2.0200000000       2.0200000000
      2.0200000000       0.0000000000       2.0200000000
      2.0200000000       2.0200000000       0.0000000000
 in angstroms                                                      and atomic positions are       0.0000000000       0.0000000000       0.0000000000 
 in angstromsassistant: &CONTROL
  calculation = 'scf'
  etot_conv_thr =   1.0000000000d-05
  forc_conv_thr =   5.0000000000d-05
  max_seconds =   6.8400000000d+03
  outdir = './out/'
  prefix = 'aiida'
  pseudo_dir = './pseudo/'
  restart_mode = 'from_scratch'
  tprnfor = .true.
  tstress = .true.
  verbosity = 'high'
/
&SYSTEM
  degauss =   4.5000000000d-03
  ecutrho =   6.0000000000d+02
  ecutwfc =   6.0000000000d+02
  ibrav = 0
  nat = 1
  nosym = .false.
  ntyp = 1
  occupations = 'smearing'
  smearing = 'fd'
/
&ELECTRONS
 

In [ ]:
inputs = tokenizer("Generate an input file for Na1Cl1 for Quantum Espresso calculations.", return_tensors="pt")

In [ ]:
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=2048)
print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

Generate an input file for Na1Cl1 for Quantum Espresso calculations.  Cell parameters are       0.0000000000       0.0000000000       0.0000000000
      0.0000000000       2.0222222222       2.0222222222
      2.0222222222       0.0000000000       2.0222222222
      2.0222222222       2.0222222222       0.0000000000
 in angstroms                                                      and atomic positions are       0.0000000000       0.0000000000       0.0000000000 
 in angstromsassistant: &CONTROL
  calculation = 'scf'
  etot_conv_thr =   1.0000000000d-05
  forc_conv_thr =   5.0000000000d-05
  max_seconds =   6.8400000000d+03
  outdir = './out/'
  prefix = 'aiida'
  pseudo_dir = './pseudo/'
  restart_mode = 'from_scratch'
  tprnfor = .true.
  tstress = .true.
  verbosity = 'high'
/
&SYSTEM
  degauss =   4.5000000000d-03
  ecutrho =   6.0000000000d+02
  ecutwfc =   6.0000000000d+02
  ibrav = 0
  nat = 1
  nosym = .false.
  ntyp = 1
  occupations = 'smearing'
  smearing = 'fd'
/
&ELECTRONS